In [8]:
# Example 21-2. record_factory.py: A simple class factory.
def record_factory(cls_name, field_names):
    try:
        field_names = field_names.replace(',', ' ').split()  # <1>
    except AttributeError:  # no .replace or .split
        pass  # assume it's already a sequence of identifiers
    field_names = tuple(field_names)  # <2>

    def __init__(self, *args, **kwargs):  # <3>
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self):  # <4>
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):  # <5>
        values = ', '.join('{}={!r}'.format(*i) for i
                           in zip(self.__slots__, self))
        return '{}({})'.format(self.__class__.__name__, values)

    cls_attrs = dict(__slots__ = field_names,  # <6>
                     __init__  = __init__,
                     __iter__  = __iter__,
                     __repr__  = __repr__)

    return type(cls_name, (object,), cls_attrs)  # <7>

Dog = record_factory('Dog', 'name weight owner')
rex = Dog('Rex', 30, 'Bob')
print("rex : ", rex)
name, weight, _ = rex
print("name, weight : ", name, ",", weight)
print("{2}'s dog weighs {1}kg".format(*rex))
rex.weight = 32
print("rex : ", rex)

rex :  Dog(name='Rex', weight=30, owner='Bob')
name, weight :  Rex , 30
Bob's dog weighs 30kg
rex :  Dog(name='Rex', weight=32, owner='Bob')


In [ ]:
# Example 21-4. model_v6.py: A class decorator
def entity(cls):  # <1>
    for key, attr in cls.__dict__.items():  # <2>
        if isinstance(attr, Validated):  # <3>
            type_name = type(attr).__name__
            attr.storage_name = '_{}#{}'.format(type_name, key)  # <4>
    return cls  # <5>

In [12]:
# Example 21-3. bulkfood_v6.py: LineItem using Quantity and NonBlank descriptors
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

import model_v6 as model

@model.entity  # <1>
class LineItem:
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

raisins = LineItem('Golden raisins', 10, 6.95)
print("dir(raisins)[:3] : ", dir(raisins)[:3])
print("LineItem.description.storage_name : ", LineItem.description.storage_name)
print("raisins.description : ", raisins.description)
print("getattr(raisins, '_NonBlank#description') : ", getattr(raisins, '_NonBlank#description'))

dir(raisins)[:3] :  ['_NonBlank#description', '_Quantity#price', '_Quantity#weight']
LineItem.description.storage_name :  _NonBlank#description
raisins.description :  Golden raisins
getattr(raisins, '_NonBlank#description') :  Golden raisins


In [1]:
# Example 21-6. evaltime.py: Write down the numbered <[N]> markers in the order they will appear in the output
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

from evalsupport import deco_alpha

print('<[1]> evaltime module start')

class ClassOne():
    print('<[2]> ClassOne body')
    def __init__(self):
        print('<[3]> ClassOne.__init__')
    def __del__(self):
        print('<[4]> ClassOne.__del__')
    def method_x(self):
        print('<[5]> ClassOne.method_x')
    class ClassTwo(object):
        print('<[6]> ClassTwo body')

@deco_alpha
class ClassThree():
    print('<[7]> ClassThree body')
    def method_y(self):
        print('<[8]> ClassThree.method_y')

class ClassFour(ClassThree):
    print('<[9]> ClassFour body')
    def method_y(self):
        print('<[10]> ClassFour.method_y')

if __name__ == '__main__':
    print('<[11]> ClassOne tests', 30 * '.')
    one = ClassOne()
    one.method_x()
    print('<[12]> ClassThree tests', 30 * '.')
    three = ClassThree()
    three.method_y()
    print('<[13]> ClassFour tests', 30 * '.')
    four = ClassFour()
    four.method_y()

print('<[14]> evaltime module end')

<[100]> evalsupport module start
<[400]> MetaAleph body
<[700]> evalsupport module end
<[1]> evaltime module start
<[2]> ClassOne body
<[6]> ClassTwo body
<[7]> ClassThree body
<[200]> deco_alpha
<[9]> ClassFour body
<[11]> ClassOne tests ..............................
<[3]> ClassOne.__init__
<[5]> ClassOne.method_x
<[12]> ClassThree tests ..............................
<[300]> deco_alpha:inner_1
<[13]> ClassFour tests ..............................
<[10]> ClassFour.method_y
<[14]> evaltime module end


In [7]:
# Metaclasses 101
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

print("'spam'.__class__ : ", 'spam'.__class__)
print("str.__class__ : ", str.__class__)

from bulkfood_v6 import LineItem
print("LineItem.__class__ : ", LineItem.__class__)
print("type.__class__ : ", type.__class__)


'spam'.__class__ :  <class 'str'>
str.__class__ :  <class 'type'>
LineItem.__class__ :  <class 'type'>
type.__class__ :  <class 'type'>


In [9]:
# Metaclasses 101
import collections
print("collections.Iterable.__class__ : ", collections.Iterable.__class__)
import abc
print("abc.ABCMeta.__class__ : ", abc.ABCMeta.__class__)
print("abc.ABCMeta.__mro__ : ", abc.ABCMeta.__mro__)

collections.Iterable.__class__ :  <class 'abc.ABCMeta'>
abc.ABCMeta.__class__ :  <class 'type'>
abc.ABCMeta.__mro__ :  (<class 'abc.ABCMeta'>, <class 'type'>, <class 'object'>)


In [10]:
# Example 21-10. evaltime_meta.py: ClassFive is an instance of the MetaAleph metaclass.
from evalsupport import deco_alpha
from evalsupport import MetaAleph

print('<[1]> evaltime_meta module start')

@deco_alpha
class ClassThree():
    print('<[2]> ClassThree body')
    def method_y(self):
        print('<[3]> ClassThree.method_y')

class ClassFour(ClassThree):
    print('<[4]> ClassFour body')
    def method_y(self):
        print('<[5]> ClassFour.method_y')


class ClassFive(metaclass=MetaAleph):
    print('<[6]> ClassFive body')
    def __init__(self):
        print('<[7]> ClassFive.__init__')
    def method_z(self):
        print('<[8]> ClassFive.method_y')

class ClassSix(ClassFive):
    print('<[9]> ClassSix body')
    def method_z(self):
        print('<[10]> ClassSix.method_y')

if __name__ == '__main__':
    print('<[11]> ClassThree tests', 30 * '.')
    three = ClassThree()
    three.method_y()
    print('<[12]> ClassFour tests', 30 * '.')
    four = ClassFour()
    four.method_y()
    print('<[13]> ClassFive tests', 30 * '.')
    five = ClassFive()
    five.method_z()
    print('<[14]> ClassSix tests', 30 * '.')
    six = ClassSix()
    six.method_z()

print('<[15]> evaltime_meta module end')

<[1]> evaltime_meta module start
<[2]> ClassThree body
<[200]> deco_alpha
<[4]> ClassFour body
<[6]> ClassFive body
<[500]> MetaAleph.__init__
<[9]> ClassSix body
<[500]> MetaAleph.__init__
<[11]> ClassThree tests ..............................
<[300]> deco_alpha:inner_1
<[12]> ClassFour tests ..............................
<[5]> ClassFour.method_y
<[13]> ClassFive tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2
<[14]> ClassSix tests ..............................
<[7]> ClassFive.__init__
<[600]> MetaAleph.__init__:inner_2
<[15]> evaltime_meta module end


In [11]:
# Example 21-12. evalsupport.py: Definition of the metaclass MetaAleph from Example 21-7.
class MetaAleph(type):
    print('<[400]> MetaAleph body')
    def __init__(cls, name, bases, dic):
        print('<[500]> MetaAleph.__init__')
        def inner_2(self):
            print('<[600]> MetaAleph.__init__:inner_2')
        cls.method_z = inner_2

<[400]> MetaAleph body


In [12]:
# Example 21-14. bulkfood_v7.py: Inheriting from model.Entity can work, if a metaclass is behind the scene.
import model_v7 as model

class LineItem(model.Entity):  # <1>
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price

In [ ]:
# Example 21-14. - entity model
class EntityMeta(type):
    """Metaclass for business entities with validated fields"""
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)  # <1>
        for key, attr in attr_dict.items():  # <2>
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)

class Entity(metaclass=EntityMeta):  # <3>
    """Business entity with validated fields"""

In [ ]:
# Example 21-16. model_v8.py: The EntityMeta metaclass uses __prepare__, 
# and Entity now has a field_names class method

class EntityMeta(type):
    """Metaclass for business entities with validated fields"""
    @classmethod
    def __prepare__(cls, name, bases):
        return collections.OrderedDict()  # <1>
    def __init__(cls, name, bases, attr_dict):
        super().__init__(name, bases, attr_dict)
        cls._field_names = []  # <2>
        for key, attr in attr_dict.items():  # <3>
            if isinstance(attr, Validated):
                type_name = type(attr).__name__
                attr.storage_name = '_{}#{}'.format(type_name, key)
                cls._field_names.append(key)  # <4>

class Entity(metaclass=EntityMeta):
    """Business entity with validated fields"""
    @classmethod
    def field_names(cls):  # <5>
        for name in cls._field_names:
            yield name

In [19]:
import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')

import model_v8 as model

class LineItem(model.Entity):
    description = model.NonBlank()
    weight = model.Quantity()
    price = model.Quantity()
    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price
    def subtotal(self):
        return self.weight * self.price


for name in LineItem.field_names():
    print(name)

description
weight
price


In [ ]:
print(" : ", )

import sys
sys.path.append('/home/kwol/git/kw/jupyter/Fluent Python/module')